<a href="https://colab.research.google.com/github/mihnguyen/udemy-customer-analytics/blob/main/purchase-prediction/notebook_Purchase_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import pickle

In [2]:
from google.colab import files
uploaded = files.upload()

Saving purchase data.csv to purchase data.csv


In [3]:
import io

df_purchase = pd.read_csv(io.BytesIO(uploaded['purchase data.csv']), index_col = 0)  # Making ID column an index column

### Data Exploration

In [14]:
df_purchase.head()

,Day,Incidence,Brand,Quantity,Last_Inc_Brand,Last_Inc_Quantity,Price_1,Price_2,Price_3,Price_4,Price_5,Promotion_1,Promotion_2,Promotion_3,Promotion_4,Promotion_5,Sex,Marital status,Age,Education,Income,Occupation,Settlement size
ID,,,,,,,,,,,,,,,,,,,,,,,
200000001,1,0,0,0,0,0,1.59,1.87,2.01,2.09,2.66,0,1,0,0,0,0,0,47,1,110866,1,0
200000001,11,0,0,0,0,0,1.51,1.89,1.99,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0
200000001,12,0,0,0,0,0,1.51,1.89,1.99,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0
200000001,16,0,0,0,0,0,1.52,1.89,1.98,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0
200000001,18,0,0,0,0,0,1.52,1.89,1.99,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0


Check for missing values:

In [15]:
df_purchase.isnull().sum()

Day                  0
Incidence            0
Brand                0
Quantity             0
Last_Inc_Brand       0
Last_Inc_Quantity    0
Price_1              0
Price_2              0
Price_3              0
Price_4              0
Price_5              0
Promotion_1          0
Promotion_2          0
Promotion_3          0
Promotion_4          0
Promotion_5          0
Sex                  0
Marital status       0
Age                  0
Education            0
Income               0
Occupation           0
Settlement size      0
dtype: int64

### Import Customer Segmentation Model

In previous Customer Segmentation notebook, we have exported the standardized data, PCA, and K-Means PCA to be used in this Purchase Prediction analysis. Let's load those here:

In [9]:
import pickle
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/Colab Notebooks"
infile = open(DATA_PATH+'/pca.pickle','rb')
pca = pickle.load(infile)

pca

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [10]:
import pickle
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/Colab Notebooks"
infile = open(DATA_PATH+'/scaler.pickle','rb')
scaler = pickle.load(infile)

scaler

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


StandardScaler(copy=True, with_mean=True, with_std=True)

In [11]:
import pickle
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/Colab Notebooks"
infile = open(DATA_PATH+'/kmeans_pca.pickle','rb')
kmeans_pca = pickle.load(infile)

kmeans_pca

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=4, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

###Standardize Purchase Data

Apply standardized customer data to purchase data:

In [27]:
features = df_purchase[['Sex', 'Marital status', 'Age', 'Education', 'Income', 'Occupation', 'Settlement size']] # Append 7 features from customer data to purchase data
df_purchase_segm_std = scaler.transform(features) # transform() vs fit_transform(): the former uses previously computed 

###PCA

Similarly, apply standardized PCA from customer segmentation to purchase data:

In [28]:
df_purchase_segm_pca = pca.transform(df_purchase_segm_std)

###K-Means PCA

In [29]:
purchase_segm_kmeans_pca = kmeans_pca.predict(df_purchase_segm_pca)

In [30]:
df_purchase_predictors = df_purchase.copy()

In [31]:
df_purchase_predictors['Segment'] = purchase_segm_kmeans_pca # Add column Segment